# trading_with_RL

## Trading with Reinforcement Learning

## Assign input parameters

In [1]:
ticker, ws, iter_count = 'JPY=X', 20, 30
ranged = '5d'
interval = '15m'

In [2]:
import sys
pkgTF = !pip freeze | grep ^keras==
if not pkgTF:
  ! sudo pip install keras
! mkdir -p models

You are using pip version 19.0.2, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# 1. Define Agent

In [3]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam

import numpy as np
import random
from collections import deque

class Agent:
	def __init__(self, state_size, is_eval=False, model_name=""):
		self.state_size = state_size # normalized previous days
		self.action_size = 3 # sit, buy, sell
		self.memory = deque(maxlen=1000)
		self.inventory = []
		self.model_name = model_name
		self.is_eval = is_eval

		self.gamma = 0.95
		self.epsilon = 1.0
		self.epsilon_min = 0.01
		self.epsilon_decay = 0.995

		self.model = load_model("models/" + model_name) if is_eval else self._model()

	def _model(self):
		model = Sequential()
		model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
		model.add(Dense(units=32, activation="relu"))
		model.add(Dense(units=8, activation="relu"))
		model.add(Dense(self.action_size, activation="linear"))
		model.compile(loss="mse", optimizer=Adam(lr=0.001))

		return model

	def act(self, state):
		if not self.is_eval and random.random() <= self.epsilon:
			return random.randrange(self.action_size)

		options = self.model.predict(state)
		return np.argmax(options[0])

	def expReplay(self, batch_size):
		mini_batch = []
		l = len(self.memory)
		for i in range(l - batch_size + 1, l):
			mini_batch.append(self.memory[i])

		for state, action, reward, next_state, done in mini_batch:
			target = reward
			if not done:
				target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

			target_f = self.model.predict(state)
			target_f[0][action] = target
			self.model.fit(state, target_f, epochs=1, verbose=0)

		if self.epsilon > self.epsilon_min:
			self.epsilon *= self.epsilon_decay

Using TensorFlow backend.


#2. Define Functions

In [4]:
import numpy as np
import math
import pandas as pd
import datetime

def psd(ticker = "JPY=X", interval = '1m', length = '4d'):
  np.random.seed(7)
  # load the dataset
  # df = pd.read_csv('spy.csv')
  # load data from yahoo
  ticker='JPY=X'
  urx="https://query1.finance.yahoo.com/v8/finance/chart/{}?region=US&lang=en-US&includePrePost=true&interval={}&range={}"
  #period2=(datetime.datetime.now()-datetime.timedelta(1)).strftime('%s')
  url = urx.format(ticker,interval,length)
  jTmp = pd.read_json(url)['chart']['result'][0]
  pbdatetime = [ datetime.datetime.fromtimestamp(int(x)) for x in jTmp['timestamp'] ]
  df=pd.DataFrame(jTmp['indicators']['quote'][0])
  df['price']=df['close']
  df['datetime']=pbdatetime
  df.dropna(inplace=True)
  return df

# prints formatted price
def formatPrice(p):
	return "{:.2f}".format(p)

# returns the vector containing stock data from a fixed file
def getStockDataVec(ticker,wn=10,interval='1m',ranged='4d'):
	datax = psd(ticker,interval=interval,length=ranged)
	p = datax['close'].values
	pchg = datax['close'].pct_change().dropna().values
	end = len(pchg)-len(pchg)%(wn)
	p = p[-end:]
	pchg = pchg[-end:]
	return p,pchg

# returns the sigmoid
def sigmoid(x):
	return 1 / (1 + math.exp(-x))

# returns an an n-day state representation ending at time t
def getState(data, t, n):
	d = t - n + 1
	block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
	res = []
	for i in range(n - 1):
		res.append(sigmoid(block[i + 1] - block[i]))

	return np.array([res])

def getState(data, p, n):
	d=np.array([data[p:n+p]])
	return d

#3. Train Model


In [5]:
#from agent.agent import Agent
#from functions import *
#import sys

#if len(sys.argv) != 4:
	#print("Usage: python train.py [stock] [window] [episodes]")
	#exit()

stock_name, window_size, episode_count = ticker, ws, iter_count

agent = Agent(window_size)

data,datp = getStockDataVec(stock_name,wn=window_size,interval=interval,ranged=ranged)
train_ratio = int(0.8*len(data))

train_data = data[0:train_ratio]
test_data = data[train_ratio:]

l = len(train_data) - window_size
batch_size = 32

for e in range(episode_count + 1):
	print("Episode " + str(e) + "/" + str(episode_count)+ "================================")
	#state = getStateORIG(datp, 0, window_size+1 )
	state = getState(train_data, 0, window_size )

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		#print(t, l)
    
		action = agent.act(state)

		# sit
		#next_state = getStateORIG(datp, t + 1, window_size+1 )
		next_state = getState(train_data, t + 1, window_size )
		reward = 0

		if action == 1: # buy
			agent.inventory.append(train_data[t])
			print("Buy: " + formatPrice(train_data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(train_data[t] - bought_price, 0)
			total_profit += train_data[t] - bought_price
			print("Sell: " + formatPrice(train_data[t]) + " | Profit: " + formatPrice(train_data[t] - bought_price))

		done = True if t == l - 1 else False
		agent.memory.append((state, action, reward, next_state, done))
		state = next_state

		if done:
			print("--------------------------------")
			print("Total Profit: " + formatPrice(total_profit))
			print("--------------------------------")

		if len(agent.memory) > batch_size:
			agent.expReplay(batch_size)
     
	#if e % 10 == 0:
	agent.model.save("models/model_ep" + str(e))

Instructions for updating:
Colocations handled automatically by placer.
Episode 0/30================================
Buy: 107.38
Sell: 107.40 | Profit: 0.03
Buy: 107.44
Sell: 107.48 | Profit: 0.04
Buy: 107.41
Buy: 107.39
Sell: 107.36 | Profit: -0.05
Buy: 107.37
Sell: 107.38 | Profit: -0.01
Buy: 107.36
Sell: 107.37 | Profit: 0.00
Sell: 107.34 | Profit: -0.02
Instructions for updating:
Use tf.cast instead.
Buy: 107.33
Buy: 107.29
Buy: 107.29
Buy: 107.25
Buy: 107.29
Buy: 107.32
Sell: 107.38 | Profit: 0.04
Buy: 107.43
Sell: 107.41 | Profit: 0.12
Sell: 107.46 | Profit: 0.18
Buy: 107.46
Sell: 107.37 | Profit: 0.12
Buy: 107.34
Buy: 107.28
Buy: 107.32
Sell: 107.36 | Profit: 0.07
Sell: 107.38 | Profit: 0.06
Sell: 107.34 | Profit: -0.09
Buy: 107.35
Buy: 107.35
Buy: 107.31
Buy: 107.31
Sell: 107.31 | Profit: -0.15
Buy: 107.35
Sell: 107.33 | Profit: -0.02
Buy: 107.35
Sell: 107.30 | Profit: 0.03
Sell: 107.29 | Profit: -0.03
Buy: 107.29
Sell: 107.30 | Profit: -0.05
Buy: 107.29
Buy: 107.36
Buy: 107.33

Buy: 107.38
Buy: 107.44
Buy: 107.48
Buy: 107.39
Buy: 107.44
Buy: 107.42
Sell: 107.43 | Profit: 0.05
Sell: 107.43 | Profit: -0.02
Sell: 107.44 | Profit: -0.04
Sell: 107.41 | Profit: 0.02
Sell: 107.41 | Profit: -0.03
Sell: 107.44 | Profit: 0.01
Buy: 107.29
Sell: 107.32 | Profit: 0.03
Buy: 107.38
Sell: 107.43 | Profit: 0.05
Buy: 107.41
Sell: 107.40 | Profit: -0.01
Buy: 107.46
Sell: 107.46 | Profit: -0.00
Buy: 107.33
Buy: 107.35
Sell: 107.32 | Profit: -0.00
Buy: 107.30
Buy: 107.29
Sell: 107.32 | Profit: -0.03
Sell: 107.30 | Profit: -0.01
Sell: 107.28 | Profit: -0.01
Buy: 107.30
Buy: 107.29
Buy: 107.36
Sell: 107.33 | Profit: 0.03
Sell: 107.32 | Profit: 0.03
Buy: 107.33
Sell: 107.33 | Profit: -0.03
Sell: 107.36 | Profit: 0.03
Buy: 106.95
Sell: 106.99 | Profit: 0.04
Buy: 107.04
Buy: 107.03
Buy: 106.97
Buy: 107.01
Buy: 107.04
Sell: 106.99 | Profit: -0.05
Sell: 107.03 | Profit: -0.00
Sell: 107.12 | Profit: 0.16
Sell: 107.07 | Profit: 0.06
Sell: 107.08 | Profit: 0.04
Buy: 106.92
Sell: 106.89 | P

--------------------------------
Total Profit: 0.47
--------------------------------
Episode 7/30================================
Buy: 107.40
Buy: 107.40
Buy: 107.42
Buy: 107.42
Buy: 107.43
Sell: 107.44 | Profit: 0.04
Sell: 107.43 | Profit: 0.03
Sell: 107.44 | Profit: 0.02
Sell: 107.48 | Profit: 0.06
Sell: 107.39 | Profit: -0.04
Buy: 107.36
Sell: 107.37 | Profit: 0.00
Buy: 107.43
Buy: 107.41
Buy: 107.40
Sell: 107.44 | Profit: 0.01
Buy: 107.46
Sell: 107.46 | Profit: 0.05
Sell: 107.46 | Profit: 0.06
Sell: 107.40 | Profit: -0.06
Buy: 107.32
Sell: 107.31 | Profit: -0.01
Buy: 107.30
Sell: 107.28 | Profit: -0.02
Buy: 107.36
Buy: 107.33
Sell: 107.32 | Profit: -0.04
Sell: 107.33 | Profit: -0.00
Buy: 106.83
Buy: 106.82
Sell: 106.89 | Profit: 0.06
Sell: 106.91 | Profit: 0.09
Buy: 107.06
Sell: 107.09 | Profit: 0.03
Buy: 107.05
Buy: 106.99
Sell: 107.01 | Profit: -0.04
Sell: 106.92 | Profit: -0.07
Buy: 107.25
Sell: 107.24 | Profit: -0.01
Buy: 107.17
Sell: 107.18 | Profit: 0.01
Buy: 107.46
Buy: 107.

Buy: 107.33
Sell: 107.36 | Profit: 0.03
Buy: 107.14
Sell: 107.11 | Profit: -0.03
Buy: 106.99
Sell: 106.97 | Profit: -0.01
Buy: 107.00
Buy: 107.04
Sell: 107.03 | Profit: 0.04
Sell: 107.03 | Profit: -0.01
Buy: 106.98
Sell: 106.94 | Profit: -0.04
Buy: 106.92
Sell: 106.94 | Profit: 0.02
Buy: 107.13
Sell: 107.16 | Profit: 0.03
Buy: 107.37
Sell: 107.35 | Profit: -0.01
Buy: 107.46
Buy: 107.44
Sell: 107.49 | Profit: 0.03
Sell: 107.43 | Profit: -0.01
Buy: 107.46
Sell: 107.44 | Profit: -0.02
Buy: 107.46
Sell: 107.48 | Profit: 0.02
Buy: 107.68
Sell: 107.70 | Profit: 0.02
Buy: 107.67
Sell: 107.61 | Profit: -0.06
Buy: 107.70
Sell: 107.76 | Profit: 0.06
Buy: 107.77
Sell: 107.79 | Profit: 0.02
Buy: 107.74
Buy: 107.72
Sell: 107.76 | Profit: 0.02
Sell: 107.74 | Profit: 0.02
Buy: 107.99
Buy: 107.99
Buy: 107.94
Buy: 107.83
Buy: 107.87
Sell: 107.82 | Profit: -0.17
Buy: 107.84
Sell: 107.87 | Profit: -0.13
Sell: 107.85 | Profit: -0.10
Sell: 107.82 | Profit: -0.01
Sell: 107.84 | Profit: -0.03
Sell: 107.86 | 

Sell: 107.00 | Profit: -0.08
Buy: 106.97
Sell: 107.02 | Profit: 0.05
Buy: 107.16
Buy: 107.19
Buy: 107.18
Buy: 107.17
Buy: 107.15
Buy: 107.16
Sell: 107.22 | Profit: 0.06
Sell: 107.21 | Profit: 0.02
Sell: 107.21 | Profit: 0.03
Sell: 107.19 | Profit: 0.03
Sell: 107.17 | Profit: 0.02
Sell: 107.17 | Profit: 0.01
Buy: 107.47
Buy: 107.47
Buy: 107.42
Sell: 107.46 | Profit: -0.01
Sell: 107.44 | Profit: -0.03
Sell: 107.70 | Profit: 0.28
Buy: 107.66
Buy: 107.76
Sell: 107.77 | Profit: 0.11
Sell: 107.79 | Profit: 0.04
Buy: 107.74
Sell: 107.67 | Profit: -0.06
Buy: 107.97
Sell: 107.98 | Profit: 0.01
Buy: 107.78
Buy: 107.75
Buy: 107.74
--------------------------------
Total Profit: 0.40
--------------------------------
Episode 18/30================================
Buy: 107.40
Sell: 107.38 | Profit: -0.02
Buy: 107.46
Sell: 107.46 | Profit: 0.00
Buy: 107.45
Buy: 107.49
Sell: 107.43 | Profit: -0.01
Buy: 107.46
Buy: 107.44
Buy: 107.46
Buy: 107.48
Sell: 107.48 | Profit: -0.02
Buy: 107.45
Sell: 107.45 | Pro

Buy: 107.67
Sell: 107.64 | Profit: -0.03
Buy: 107.75
Sell: 107.74 | Profit: -0.01
Buy: 108.09
Buy: 108.06
Sell: 108.10 | Profit: 0.01
Sell: 108.07 | Profit: 0.01
Buy: 108.06
Sell: 107.97 | Profit: -0.09
Buy: 107.84
Sell: 107.87 | Profit: 0.03
Buy: 107.85
Sell: 107.82 | Profit: -0.03
Buy: 107.76
Sell: 107.82 | Profit: 0.05
--------------------------------
Total Profit: 0.21
--------------------------------
Episode 23/30================================
Buy: 107.42
Sell: 107.43 | Profit: 0.01
Buy: 107.43
Sell: 107.44 | Profit: 0.01
Buy: 107.41
Sell: 107.41 | Profit: 0.00
Buy: 107.37
Sell: 107.38 | Profit: 0.01
Buy: 107.37
Sell: 107.34 | Profit: -0.03
Buy: 107.35
Sell: 107.34 | Profit: -0.01
Buy: 107.35
Sell: 107.36 | Profit: 0.01
Buy: 107.33
Sell: 107.31 | Profit: -0.02
Buy: 107.33
Sell: 107.33 | Profit: 0.00
Buy: 107.36
Buy: 107.33
Buy: 107.31
Buy: 107.17
Buy: 107.12
Sell: 107.14 | Profit: -0.22
Buy: 107.12
Buy: 107.04
Buy: 107.06
Buy: 107.05
Sell: 107.04 | Profit: -0.29
Sell: 106.95 | P

Buy: 106.98
Sell: 106.97 | Profit: -0.01
Buy: 106.94
Sell: 106.94 | Profit: 0.00
Buy: 106.92
Sell: 106.91 | Profit: -0.01
Buy: 107.16
Buy: 107.14
Buy: 107.13
Sell: 107.16 | Profit: 0.01
Sell: 107.19 | Profit: 0.05
Sell: 107.17 | Profit: 0.04
Buy: 107.30
Sell: 107.37 | Profit: 0.06
Buy: 107.46
Buy: 107.44
Buy: 107.70
Sell: 107.63 | Profit: 0.17
Sell: 107.68 | Profit: 0.24
Sell: 107.68 | Profit: -0.02
Buy: 107.65
Sell: 107.64 | Profit: -0.01
Buy: 107.78
Buy: 107.72
Buy: 107.71
Sell: 107.73 | Profit: -0.05
Sell: 107.75 | Profit: 0.03
Sell: 107.75 | Profit: 0.04
Buy: 107.71
Sell: 107.79 | Profit: 0.08
Buy: 107.87
Buy: 107.84
Buy: 107.84
Buy: 107.87
Sell: 107.90 | Profit: 0.03
Sell: 107.88 | Profit: 0.04
Sell: 107.85 | Profit: 0.02
Sell: 107.87 | Profit: -0.00
--------------------------------
Total Profit: 0.57
--------------------------------
Episode 28/30================================
Buy: 107.39
Buy: 107.44
Buy: 107.42
Sell: 107.41 | Profit: 0.02
Buy: 107.44
Buy: 107.39
Sell: 107.36 | 

#4. Testing or Evaluating

In [14]:
import keras
from keras.models import load_model

#from agent.agent import Agent
#from functions import *
#import sys

#if len(sys.argv) != 3:
	#print("Usage: python evaluate.py [stock] [model]")
	#exit()

model_name = 'model_ep1'
model = load_model('models/'+model_name)
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent(window_size, True, model_name)
data,datp = getStockDataVec(stock_name,wn=window_size)
l = len(data) - window_size
batch_size = 32

state = getState(data, 0, window_size)
total_profit = 0
agent.inventory = []

for t in range(l):
	action = agent.act(state)

	# sit
	next_state = getState(data, t + 1, window_size)
	reward = 0

	if action == 1: # buy
		agent.inventory.append(data[t])
		print("Buy: " + formatPrice(data[t]))

	elif action == 2 and len(agent.inventory) > 0: # sell
		bought_price = agent.inventory.pop(0)
		reward = max(data[t] - bought_price, 0)
		total_profit += data[t] - bought_price
		print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

	done = True if t == l - 1 else False
	agent.memory.append((state, action, reward, next_state, done))
	state = next_state

	if done:
		print("--------------------------------")
		print(stock_name + " Total Profit: " + formatPrice(total_profit))
		print("--------------------------------")

Buy: 107.31
Buy: 107.31
Buy: 107.31
Buy: 107.31
Buy: 107.32
Buy: 107.33
Buy: 107.33
Buy: 107.33
Buy: 107.32
Buy: 107.33
Buy: 107.32
Buy: 107.33
Buy: 107.33
Buy: 107.33
Buy: 107.34
Buy: 107.33
Buy: 107.33
Buy: 107.33
Buy: 107.33
Buy: 107.33
Buy: 107.34
Buy: 107.35
Buy: 107.36
Buy: 107.35
Buy: 107.35
Buy: 107.35
Buy: 107.35
Buy: 107.34
Buy: 107.33
Buy: 107.33
Buy: 107.33
Buy: 107.33
Buy: 107.33
Buy: 107.32
Buy: 107.34
Buy: 107.33
Buy: 107.32
Buy: 107.31
Buy: 107.32
Buy: 107.32
Buy: 107.31
Buy: 107.30
Buy: 107.30
Buy: 107.31
Buy: 107.31
Buy: 107.32
Buy: 107.31
Buy: 107.32
Buy: 107.32
Buy: 107.32
Buy: 107.32
Buy: 107.33
Buy: 107.33
Buy: 107.35
Buy: 107.37
Buy: 107.38
Buy: 107.38
Buy: 107.39
Buy: 107.39
Buy: 107.37
Buy: 107.38
Buy: 107.38
Buy: 107.40
Buy: 107.39
Buy: 107.40
Buy: 107.40
Buy: 107.38
Buy: 107.37
Buy: 107.35
Buy: 107.36
Buy: 107.36
Buy: 107.37
Buy: 107.36
Buy: 107.36
Buy: 107.35
Buy: 107.36
Buy: 107.37
Buy: 107.35
Buy: 107.35
Buy: 107.33
Buy: 107.35
Buy: 107.36
Buy: 107.36
Buy:

Buy: 107.05
Buy: 107.05
Buy: 107.05
Buy: 107.05
Buy: 107.05
Buy: 107.06
Buy: 107.05
Buy: 107.05
Buy: 107.05
Buy: 107.03
Buy: 107.04
Buy: 107.04
Buy: 107.03
Buy: 107.03
Buy: 107.00
Buy: 106.99
Buy: 106.98
Buy: 106.98
Buy: 106.97
Buy: 106.97
Buy: 106.96
Buy: 106.96
Buy: 106.99
Buy: 107.01
Buy: 107.01
Buy: 107.02
Buy: 106.99
Buy: 107.01
Buy: 107.00
Buy: 107.01
Buy: 107.02
Buy: 107.02
Buy: 107.04
Buy: 107.03
Buy: 107.01
Buy: 107.03
Buy: 107.03
Buy: 107.01
Buy: 106.99
Buy: 106.99
Buy: 106.97
Buy: 106.98
Buy: 106.96
Buy: 106.94
Buy: 106.95
Buy: 106.93
Buy: 106.89
Buy: 106.88
Buy: 106.89
Buy: 106.91
Buy: 106.91
Buy: 106.94
Buy: 106.92
Buy: 106.90
Buy: 106.88
Buy: 106.85
Buy: 106.86
Buy: 106.86
Buy: 106.84
Buy: 106.85
Buy: 106.87
Buy: 106.86
Buy: 106.87
Buy: 106.88
Buy: 106.88
Buy: 106.88
Buy: 106.89
Buy: 106.89
Buy: 106.90
Buy: 106.89
Buy: 106.91
Buy: 106.90
Buy: 106.89
Buy: 106.88
Buy: 106.89
Buy: 106.91
Buy: 106.89
Buy: 106.88
Buy: 106.90
Buy: 106.90
Buy: 106.91
Buy: 106.91
Buy: 106.92
Buy:

Buy: 107.44
Buy: 107.45
Buy: 107.46
Buy: 107.45
Buy: 107.45
Buy: 107.43
Buy: 107.42
Buy: 107.42
Buy: 107.43
Buy: 107.42
Buy: 107.42
Buy: 107.43
Buy: 107.43
Buy: 107.43
Buy: 107.42
Buy: 107.43
Buy: 107.43
Buy: 107.44
Buy: 107.43
Buy: 107.43
Buy: 107.43
Buy: 107.43
Buy: 107.43
Buy: 107.43
Buy: 107.43
Buy: 107.43
Buy: 107.44
Buy: 107.44
Buy: 107.43
Buy: 107.44
Buy: 107.44
Buy: 107.44
Buy: 107.46
Buy: 107.47
Buy: 107.47
Buy: 107.49
Buy: 107.49
Buy: 107.48
Buy: 107.47
Buy: 107.48
Buy: 107.48
Buy: 107.49
Buy: 107.49
Buy: 107.49
Buy: 107.49
Buy: 107.50
Buy: 107.50
Buy: 107.48
Buy: 107.47
Buy: 107.47
Buy: 107.47
Buy: 107.47
Buy: 107.47
Buy: 107.47
Buy: 107.48
Buy: 107.48
Buy: 107.49
Buy: 107.49
Buy: 107.49
Buy: 107.49
Buy: 107.49
Buy: 107.48
Buy: 107.48
Buy: 107.47
Buy: 107.47
Buy: 107.46
Buy: 107.46
Buy: 107.46
Buy: 107.46
Buy: 107.46
Buy: 107.45
Buy: 107.46
Buy: 107.46
Buy: 107.46
Buy: 107.47
Buy: 107.47
Buy: 107.47
Buy: 107.45
Buy: 107.45
Buy: 107.44
Buy: 107.45
Buy: 107.45
Buy: 107.44
Buy:

Buy: 107.81
Buy: 107.81
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.80
Buy: 107.80
Buy: 107.81
Buy: 107.82
Buy: 107.81
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.81
Buy: 107.80
Buy: 107.81
Buy: 107.81
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.79
Buy: 107.80
Buy: 107.79
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.79
Buy: 107.80
Buy: 107.80
Buy: 107.79
Buy: 107.81
Buy: 107.81
Buy: 107.79
Buy: 107.79
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.77
Buy: 107.78
Buy: 107.77
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.77
Buy: 107.78
Buy: 107.78
Buy: 107.77
Buy: 107.76
Buy: 107.78
Buy: 107.77
Buy: 107.78
Buy: 107.77
Buy: 107.77
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.77
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.77
Buy: 107.79
Buy:

Buy: 107.83
Buy: 107.85
Buy: 107.85
Buy: 107.84
Buy: 107.82
Buy: 107.82
Buy: 107.81
Buy: 107.81
Buy: 107.82
Buy: 107.83
Buy: 107.83
Buy: 107.84
Buy: 107.85
Buy: 107.85
Buy: 107.87
Buy: 107.87
Buy: 107.86
Buy: 107.89
Buy: 107.86
Buy: 107.87
Buy: 107.88
Buy: 107.87
Buy: 107.88
Buy: 107.88
Buy: 107.89
Buy: 107.89
Buy: 107.89
Buy: 107.89
Buy: 107.88
Buy: 107.88
Buy: 107.87
Buy: 107.88
Buy: 107.90
Buy: 107.87
Buy: 107.93
Buy: 107.90
Buy: 107.88
Buy: 107.88
Buy: 107.89
Buy: 107.89
Buy: 107.89
Buy: 107.90
Buy: 107.90
Buy: 107.91
Buy: 107.89
Buy: 107.90
Buy: 107.89
Buy: 107.90
Buy: 107.90
Buy: 107.94
Buy: 107.95
Buy: 107.93
Buy: 107.92
Buy: 107.92
Buy: 107.91
Buy: 107.91
Buy: 107.90
Buy: 107.89
Buy: 107.89
Buy: 107.87
Buy: 107.88
Buy: 107.87
Buy: 107.88
Buy: 107.88
Buy: 107.88
Buy: 107.87
Buy: 107.86
Buy: 107.87
Buy: 107.87
Buy: 107.84
Buy: 107.86
Buy: 107.85
Buy: 107.86
Buy: 107.86
Buy: 107.87
Buy: 107.87
Buy: 107.87
Buy: 107.85
Buy: 107.85
Buy: 107.83
Buy: 107.85
Buy: 107.86
Buy: 107.85
Buy:

Buy: 107.64
Buy: 107.64
Buy: 107.65
Buy: 107.65
Buy: 107.65
Buy: 107.67
Buy: 107.67
Buy: 107.66
Buy: 107.65
Buy: 107.64
Buy: 107.65
Buy: 107.64
Buy: 107.64
Buy: 107.63
Buy: 107.63
Buy: 107.63
Buy: 107.62
Buy: 107.63
Buy: 107.64
Buy: 107.63
Buy: 107.64
Buy: 107.66
Buy: 107.68
Buy: 107.67
Buy: 107.66
Buy: 107.67
Buy: 107.66
Buy: 107.66
Buy: 107.65
Buy: 107.63
Buy: 107.64
Buy: 107.64
Buy: 107.63
Buy: 107.63
Buy: 107.64
Buy: 107.65
Buy: 107.65
Buy: 107.65
Buy: 107.65
Buy: 107.64
Buy: 107.64
Buy: 107.64
Buy: 107.64
Buy: 107.62
Buy: 107.61
Buy: 107.62
Buy: 107.62
Buy: 107.62
Buy: 107.62
Buy: 107.62
Buy: 107.60
Buy: 107.62
Buy: 107.62
Buy: 107.61
Buy: 107.62
Buy: 107.62
Buy: 107.63
Buy: 107.67
Buy: 107.67
Buy: 107.66
Buy: 107.67
Buy: 107.67
Buy: 107.67
Buy: 107.67
Buy: 107.65
Buy: 107.65
Buy: 107.67
Buy: 107.64
Buy: 107.66
Buy: 107.65
Buy: 107.67
Buy: 107.68
Buy: 107.68
Buy: 107.69
Buy: 107.69
Buy: 107.68
Buy: 107.68
Buy: 107.67
Buy: 107.68
Buy: 107.68
Buy: 107.67
Buy: 107.68
Buy: 107.70
Buy:

Buy: 107.77
Buy: 107.79
Buy: 107.76
Buy: 107.77
Buy: 107.76
Buy: 107.77
Buy: 107.77
Buy: 107.78
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.77
Buy: 107.78
Buy: 107.76
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.79
Buy: 107.79
Buy: 107.78
Buy: 107.78
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.79
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.78
Buy: 107.79
Buy: 107.80
Buy: 107.80
Buy: 107.79
Buy: 107.80
Buy: 107.80
Buy: 107.79
Buy: 107.81
Buy: 107.79
Buy: 107.80
Buy: 107.80
Buy: 107.81
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.80
Buy: 107.78
Buy: 107.79
Buy: 107.80
Buy: 107.80
Buy: 107.78
Buy: 107.78
Buy: 107.80
Buy: 107.80
Buy: 107.82
Buy: 107.81
Buy: 107.81
Buy: 107.80
Buy: 107.81
Buy: 107.81
Buy: 107.80
Buy: 107.81
Buy: 107.80
Buy: 107.81
Buy: 107.82
Buy: 107.81
Buy: 107.80
Buy: 107.81
Buy: 107.80
Buy: 107.81
Buy: 107.81
Buy: 107.83
Buy: 107.82
Buy: 107.78
Buy: 107.81
Buy: 107.81
Buy: 107.81
Buy: 107.81
Buy: 107.81
Buy: